In [1]:
#dependencies
import os
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [2]:
#Setup database
app = Flask(__name__)
# app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///ffdb"
# db = SQLAlchemy(app)
ffdb = sqlite3.connect('ffdb')

# Pull data from database into a local variable
# This would be really unacceptable if our database wasn't basically a tiny little spreadsheet
# If complexity increased, it would be very reasonable to tailor select statements to specific requests inside of the functions I call below
cursor = ffdb.cursor()
cursor.execute('''
    SELECT * from fire
''')
data = cursor.fetchall()

In [ ]:
@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")


@app.route("/states")
def states():
    """Return a list of sample names."""
    
    return jsonify(['Acre', 'Alagoas', 'Amapa', 'Amazonas', 'Bahia', 'Ceara', 'Distrito Federal', 'Espirito Santo', 'Goias', 'Maranhao', 'Mato Grosso', 'Minas Gerais', 'Pará', 'Paraiba', 'Pernambuco', 'Piau', 'Rio', 'Rondonia', 'Roraima', 'Santa Catarina', 'Sao Paulo', 'Sergipe', 'Tocantins'])


@app.route("/state_data/<state>")
#  def sample_metadata(sample):
def state_data(state):
   

    """Return the Data for a given state."""
    results = []
    for x in data:
        if x[1] == state:
            results.append(x)
#     print(results)
    


    # Create a dictionary entry for each row of information
    state_data_dict = {}
    n = 0
    
    # turn each row into its own dictionary and add that to the big dictionary
    for result in results:
        state_data = {}
        state_data["year"] = result[0]
        state_data["state"] = result[1]
        state_data["month"] = result[2]
        state_data["number"] = result[3]
        state_data["date"] = result[4]
        state_data_dict[n] = state_data
        n += 1
    
#return that info jsonified
    print(state_data_dict)
    return jsonify(state_data_dict)

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2019 19:02:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2019 19:02:14] "GET /states HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2019 19:02:14] "GET /state_data/Acre HTTP/1.1" 200 -


{0: {'year': 1998, 'state': 'Acre', 'month': 'Janeiro', 'number': 0.0, 'date': '1998-01-01'}, 1: {'year': 1999, 'state': 'Acre', 'month': 'Janeiro', 'number': 0.0, 'date': '1999-01-01'}, 2: {'year': 2000, 'state': 'Acre', 'month': 'Janeiro', 'number': 0.0, 'date': '2000-01-01'}, 3: {'year': 2001, 'state': 'Acre', 'month': 'Janeiro', 'number': 0.0, 'date': '2001-01-01'}, 4: {'year': 2002, 'state': 'Acre', 'month': 'Janeiro', 'number': 0.0, 'date': '2002-01-01'}, 5: {'year': 2003, 'state': 'Acre', 'month': 'Janeiro', 'number': 10.0, 'date': '2003-01-01'}, 6: {'year': 2004, 'state': 'Acre', 'month': 'Janeiro', 'number': 0.0, 'date': '2004-01-01'}, 7: {'year': 2005, 'state': 'Acre', 'month': 'Janeiro', 'number': 12.0, 'date': '2005-01-01'}, 8: {'year': 2006, 'state': 'Acre', 'month': 'Janeiro', 'number': 4.0, 'date': '2006-01-01'}, 9: {'year': 2007, 'state': 'Acre', 'month': 'Janeiro', 'number': 0.0, 'date': '2007-01-01'}, 10: {'year': 2008, 'state': 'Acre', 'month': 'Janeiro', 'number': 0